In [1]:
# 10/20/17 Test script for NG search formulation using ipopt. 

using JuMP
using Ipopt
# using Polynomials, Interpolations

import JuMP.JuMPArray, JuMP.Variable

# function constrKin(mpc::Model,s,snum,mpc_hor)
#     # kinematic constraint (nonlinear equality)
#     z = convState[s,snum,'z']
#     u = convState[s,snum,'u']
#     N = mpc_hor
#     for ii = 1:N
#         z[:,ii+1] == z[:,ii]+ [z[4,ii]*cos[z[3,ii]];z[4,ii]*sin(z[3,ii]);u[:,ii]]*dt
#     end    
# end

# function getObj(mpc::Model,s,snum,mpc_hor,gmm_num,alp1,alp2,alp3,infovflag)
#     x = convState(s,snum,'x') #this.convState
#     u = convState(s,snum,'u')
#     P = convState(s,snum,'P')
#     z = convState(s,snum,'z')
#     Pinverse = convState(s,snum,"Pinverse")
# #     auxt = convState[s,snum,"auxt"]
# #     slkm = convState[s,snum,"slkm"]
# #     slkt = convState[s,snum,"slkt"] 

#     N = mpc_hor #this.mpc_hor
#     val = 0
#     [~,max_idx] = maximum(wt)
#     for ii = 2:N+1       
#        for jj = 1:gmm_num #this.gmm_num
#            tmp = 0
#            for ll = 1:gmm_num     
#                 val = val-10*wt[jj]*log(tmp)
#                 if ~infovflag
#                    val = val+sum((x[2*max_idx-1:2*max_idx,ii]-z[1:2,ii]).^2); # penalize the distance between sensor and MLE target postion with maximal weight
#                 end
#             end
#         end
#     end
# end

function convState(s::Any,snum::Dict,sstring::AbstractString,N::Int64)
    print(s)
    if sstring == "z"       
        t = s[snum["idx"][1,1]:snum["idx"][1,2]]
        t = reshape(t,4,N+1);                   
    elseif sstring == "u"
        t = s[snum["idx"][2,1]:snum["idx"][2,2]]
        t = reshape(t,2,N)
    elseif sstring == "x"
        t = s[snum["idx"][3,1]:snum["idx"][3,2]]
        t = reshape(t,2*this.gmm_num,N+1)
    elseif sstring == "x_pred"
       t = s[snum["idx"][4,1]:snum["idx"][4,2]]
       t = reshape(t,2*this.gmm_num,N)
    elseif sstring == "P"
       t = s[snum["idx"][5,1]:snum["idx"][5,2]]
       t = reshape(t,2,2,this.gmm_num,N+1)
    elseif sstring == "P_pred"
       t = s[snum["idx"][6,1]:snum["idx"][6,2]]
       t = reshape(t,2,2,this.gmm_num,N)
    elseif sstring == "K"
       t = s[snum["idx"][7,1]:snum["idx"][7,2]]
       t = reshape(t,2,2,this.gmm_num,N)
    elseif sstring == "Pinverse"
       t = s[snum["idx"][8,1]:snum["idx"][8,2]]
       t = reshape(t,2,2,this.gmm_num,N+1)
#        case "auxt"
#            t = s[snum.idx[9,1]:snum.idx[9,2]]
#            t = reshape(t,this.gmm_num,N)
#        case "auxm"
#            t = s[snum.idx[10,1]:snum.idx[10,2]]
#            t = reshape(t,this.gmm_num,this.gmm_num,N)
#        case "gam_var"
#            t = s[snum.idx[11,1]:snum.idx[11,2]]
#            t = reshape(t,this.gmm_num,N)
#        case "slkm"
#            t = s[snum.idx[12,1]:snum.idx[12,2]]
#            t = reshape(t,this.gmm_num,this.gmm_num,N)
#        case "slkt"
#            t = s[snum.idx[13,1]:snum.idx[13,2]]
#            t = reshape(t,this.gmm_num,N)
#        case "auxgau"
#            t = s[snum.idx[14,1]:snum.idx[14,2]]
#            t = reshape(t,this.gmm_num,this.gmm_num,N)
    
    end
    return t
end

function setState(z::Array{Float64,2},u::Array{Float64,2}) #,x::Array{Float64,2},xpred::Array{Float64,2},P::Array{Float64,4},P_pred::Array{Float64,4},K::Array{Float64,4},Pinverse::Array{Float64,4})
    s = [z[:];u[:]];#;x[:];xpred[:];P[:];P_pred[:];K[:];Pinverse[:];auxt[:];auxm[:];gam_var[:];slkm[:];slkt[:];auxgau[:]]
    
    # size of each decision variables
    lenz = length(z[:]); #4*(N+1)
    lenu = length(u[:]); #2*N
#     lenx = length(x[:]); #2*this.gmm_num*(N+1)
#     lenxpred = length(xpred[:]); #2*this.gmm_num*N
#     lenP = length(P[:]); #2*2*this.gmm_num*(N+1)
#     lenP_pred = length(P_pred[:]); #2*2*this.gmm_num*N
#     lenK = length(K[:]); #2*2*this.gmm_num*N
#     lenPinverse = length(Pinverse[:]); #2*2*this.gmm_num*N

    # compute the section of s that corresponds to different
    # decision variables
    snum = Dict();
#     snum.idx = zeros(9,2)
    snum["idx"] = floor(Int64,zeros(8,2))
    snum["idx"][1,:] = [1,lenz]
    snum["idx"][2,:] = [snum["idx"][1,2]+1,snum["idx"][1,2]+lenu]
#     snum["idx"][3,:] = [snum["idx"][2,2]+1,snum["idx"][2,2]+lenx]
#     snum["idx"][4,:] = [snum["idx"][3,2]+1,snum["idx"][3,2]+lenxpred]
#     snum["idx"][5,:] = [snum["idx"][4,2]+1,snum["idx"][4,2]+lenP]
#     snum["idx"][6,:] = [snum["idx"][5,2]+1,snum["idx"][5,2]+lenP_pred]
#     snum["idx"][7,:] = [snum["idx"][6,2]+1,snum["idx"][6,2]+lenK]
#     snum["idx"][8,:] = [snum["idx"][7,2]+1,snum["idx"][7,2]+lenPinverse]
    
    snum["total"] = snum["idx"][2,2] 
#     snum["total"] = snum["idx"][14,2] 
    return s,snum
end

### main functional part
# robot
N = 2
# init_state = [30;30;0]
# maxV = 1.5

# field
x_len = 50 # x length
y_len = 50 # y length
xp = 1:(x_len+1) # points in x direction, i.e. horizontal distance between adjacent points are 0.2
yp = 1:(y_len+1) # points in y direction
tar_pos = [5;5]
# tar_r = 3

zLB = [xp[1],yp[1]]
zUB = [xp[end],yp[end]]
uLB = [0,-pi/4]
uUB = [1.5,pi/4]
dt = 1

# initial solution
z0 = [[10.0 10.0 10.0];[10.0 10.0 10.0];[0.0 0.0 0.0];[0.0 0.0 0.0]];
u0 = [[0.0 0.0];[0.0 0.0]];
nz = 4;
nu = 2;
# s,snum = setState(z0,u0);

# formulate MPC problem
mpc = Model(solver=IpoptSolver(max_iter=1000, max_cpu_time=100.0, print_timing_statistics="yes")) #max_cpu_time=dtMPC, hessian_approximation ="limited-memory"

# define variables
@variable(mpc,z[1:nz,1:N+1])
@variable(mpc,u[1:nu,1:N])
# @variable(mpc,sp[length(s),1])
# z = convState(sp,snum,"z",N)
# u = convState(sp,snum,"u",N)
# x = convState(s,snum,'x',N) #this.convState
# xpred = convState(s,snum,'x_pred',N)
# P = convState(s,snum,'P',N)
# Ppred = convState(s,snum,'P_pred',N)
# K = convState(s,snum,'K',N)
# Pinverse = convState(s,snum,"Pinverse",N)



# objective
# getObj

# @NLexpression(mpc,val, z[1,1]^4)

# [~,max_idx] = maximum(wt)
# for ii = 2:N+1       
#    for jj = 1:gmm_num #this.gmm_num
#        tmp = 0
#        for ll = 1:gmm_num     
#             val = val-10*wt[jj]*log(tmp)
#             if ~infovflag
#                val = val+sum((x[2*max_idx-1:2*max_idx,ii]-z[1:2,ii]).^2); # penalize the distance between sensor and MLE target postion with maximal weight
#             end
#         end
#     end
# end

@NLexpression(mpc,[jj=1:gmm_num,ll=1:gmm_num], )## resume from here
@NLobjective(mpc,Min,val)

### constraints
# constrLinIneq

# constrLinEq

# constrKin
# kinematic constraint (nonlinear equality)
@NLconstraint(mpc,kinConstr_x[ii = 1:N],z[1,ii+1] == z[1,ii]+ z[4,ii]*cos(z[3,ii])*dt)
@NLconstraint(mpc,kinConstr_y[ii = 1:N],z[2,ii+1] == z[2,ii]+ z[4,ii]*sin(z[3,ii])*dt)
@NLconstraint(mpc,kinConstr_theta[ii = 1:N],z[3,ii+1] == z[3,ii]+ u[1,ii]*dt)
@NLconstraint(mpc,kinConstr_v[ii = 1:N],z[4,ii+1] == z[4,ii]+ u[2,ii]*dt)

# constrBel

# constrPinverse

# solve problem
status = solve(mpc)
print(status)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       28
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       13

Total number of variables............................:       16
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

Optimal[0.0 0.0 0.0
 0.0 0.0 0.0
 0.0 0.0 0.0
 0.0 0.0 0.0][0.0 0.0
 0.0 0.0]